In [1]:
import cv2
import matplotlib.pyplot as plt
import image_dehazer
import numpy as np
from threading import Thread

In [2]:
def cropping(img):
    height = img.shape[0]
    width = img.shape[1]
    new_h = height//4
    new_w = width//4
    new_img = img[new_h:int(new_h*3),new_w:int(new_w*3)]
    return new_img

In [3]:
def LIME(img):
#     img = cv2.resize(img, (60,80))

    #Invert the image
    new_img = 255-img
    
    #Dehazing the inverted image
    dehaze_img = image_dehazer.remove_haze(new_img)


    #inverse the dehaze image
    inv_img = 255-dehaze_img

    
    return inv_img

In [4]:
class VideoGet:
    """
    Class that continuously gets frames from a VideoCapture object
    with a dedicated thread.
    """

    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()
        self.stopped = False

    def start(self):    
        Thread(target=self.get, args=()).start()
        return self

    def get(self):
        while not self.stopped:
            if not self.grabbed:
                self.stop()
            else:
                (self.grabbed, self.frame) = self.stream.read()
                

    def stop(self):
        self.stopped = True

In [5]:
# from datetime import datetime

# class CountsPerSec:
#     """
#     Class that tracks the number of occurrences ("counts") of an
#     arbitrary event and returns the frequency in occurrences
#     (counts) per second. The caller must increment the count.
#     """

#     def __init__(self):
#         self._start_time = None
#         self._num_occurrences = 0

#     def start(self):
#         self._start_time = datetime.now()
#         return self

#     def increment(self):
#         self._num_occurrences += 1

#     def countsPerSec(self):
#         elapsed_time = (datetime.now() - self._start_time).total_seconds()
#         return self._num_occurrences / elapsed_time if elapsed_time > 0 else 0

In [6]:
# def putIterationsPerSec(frame, iterations_per_sec):
#     """
#     Add iterations per second text to lower-left corner of a frame.
#     """

#     cv2.putText(frame, "{:.0f} iterations/sec".format(iterations_per_sec),
#         (10, 450), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255,0),3)
#     return frame


In [7]:
class VideoShow:
    """
    Class that continuously shows a frame using a dedicated thread.
    """

    def __init__(self, frame=None):
        self.frame = frame
        self.stopped = False

    def start(self):
        Thread(target=self.show, args=()).start()
        return self

    def show(self):
        while not self.stopped:
            cv2.imshow("Video", self.frame)
            if cv2.waitKey(1) == ord("q"):
                self.stopped = True

    def stop(self):
        self.stopped = True

In [8]:
# import tensorflow
# from tensorflow.keras import models
# mbllen = models.load_model('Syn_img_lowlight_model.h5')
# def mbllen_model(frame):
#     img_A = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
#     img_A = img_A/255
#     img_A = img_A[np.newaxis, :]
    
#     width = img_A.shape[1]
#     height = img_A.shape[2]
    
#     pred_img = mbllen.predict(img_A)
    
#     return pred_img.reshape(width,height,3)

In [9]:
eye_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
def drowsy(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_classifier.detectMultiScale(gray,1.3,4)
    if len(eyes) == 0:
        print("eyes are not detected")
    else:
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(frame,(ex,ey),(ex+ew,ey+eh),(255,255,0),2)
    return frame

In [10]:
# from datetime import datetime
# class fps():
#     def __init__(self):
#         self.inc=0
#         self.s = datetime.now()
#         self.s_m = self.s.strftime("%M")
#         self.s_s = self.s.strftime("%S")
#         self.start_time = int(self.s_m)*60+int(self.s_s)
        
#     def increment(self):
#         self.inc+=1
        
#     def fps_calculate(self):
#         return self.start_time, self.end_time, self.inc/(self.end_time-self.start_time)
    
#     def spf_calculate(self):
#         return (self.end_time-self.start_time)/self.inc
    
#     def stop(self):
#         self.e = datetime.now()
#         self.e_m = self.e.strftime("%M")
#         self.e_s = self.e.strftime("%S")
#         self.end_time =int(self.e_m)*60+int(self.e_s)

In [11]:
def threadBoth():
    """
    Dedicated thread for grabbing video frames with VideoGet object.
    Dedicated thread for showing video frames with VideoShow object.
    Main thread serves only to pass frames between VideoGet and
    VideoShow objects/threads.
    """

    video_getter = VideoGet(0).start()
    video_shower = VideoShow(video_getter.frame).start()
    
    while True:
        if video_getter.stopped or video_shower.stopped:
            video_shower.stop()
            video_getter.stop()
            break

        frame = video_getter.frame
        
        #cropping image
        frame = cropping(frame)
    
        #Low-light-image-enhancement
        inv_img = LIME(frame)

        #detect eye
        new_img = drowsy(inv_img)
        
#         f.increment()
        video_shower.frame = inv_img

In [12]:
# f=fps()
threadBoth()

eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are not detected
eyes are n

In [13]:
# f.stop()

In [14]:
# print('fps',str(f.fps_calculate()))
# print('spf',str(f.spf_calculate()))

In [15]:
# f.fps_calculate()

In [16]:
# f.spf_calculate()